In [87]:
from wildlife_datasets import datasets
from wildlife_tools.data.dataset import WildlifeDataset
import torchvision.transforms as T

In [ ]:
datasets = [
    "OpenCows2020",
    "ATRW",
    "BelugaID",
    "CTai",
    "GiraffeZebraID",
    "Giraffes",
    "HumpbackWhaleID",
    "HyenaID2022",
    "IPanda50",
    "LeopardID2022",
    "MacaqueFaces",
    "NyalaData",
    "SealID",
    "SeaTurtleID",
    "StripeSpotter",
    "WhaleSharkID",
    "ZindiTurtleRecall"
]

In [80]:
datasets.OpenCows2020.get_data('datasets/OpenCows2020')

DATASET OpenCows2020: DOWNLOADING STARTED.


10m32xl88x2b61zlkkgz3fml17.zip: 2.27GB [03:06, 12.2MB/s]                               


DATASET OpenCows2020: EXTRACTING STARTED.
DATASET OpenCows2020: FINISHED.



In [86]:
datasets.

[wildlife_datasets.datasets.datasets.GreenSeaTurtles,
 wildlife_datasets.datasets.datasets.SeaTurtleIDHeads,
 wildlife_datasets.datasets.datasets.ZindiTurtleRecall]

In [82]:
root = 'datasets/OpenCows2020'

d = datasets.OpenCows2020(root)

d.df

,image_id,identity,path,original_split
0,00192639fd6df155,016,10m32xl88x2b61zlkkgz3fml17/identification/imag...,train
1,001d0519bc54343e,013,10m32xl88x2b61zlkkgz3fml17/identification/imag...,train
2,002c48f3caf7516e,004,10m32xl88x2b61zlkkgz3fml17/identification/imag...,train
3,00517a4f8038ee4b,012,10m32xl88x2b61zlkkgz3fml17/identification/imag...,test
4,00538b3d4b88aed0,010,10m32xl88x2b61zlkkgz3fml17/identification/imag...,train
...,...,...,...,...
4731,ffe12713a0364f02,006,10m32xl88x2b61zlkkgz3fml17/identification/imag...,test
4732,fff2b100edabf605,004,10m32xl88x2b61zlkkgz3fml17/identification/imag...,train
4733,fff5d5e9763a1a93,030,10m32xl88x2b61zlkkgz3fml17/identification/imag...,train
4734,fff7204a24d58cdc,008,10m32xl88x2b61zlkkgz3fml17/identification/imag...,train


In [83]:
import torchvision.transforms as T 
from wildlife_datasets import datasets
from wildlife_tools.data.dataset import WildlifeDataset
from wildlife_tools.data.split import SplitMetadata

root = 'datasets/OpenCows2020'

d = datasets.OpenCows2020(root)

transform = T.Compose([
    T.Resize(size=(224, 224)),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

reference_dataset = WildlifeDataset(
    metadata = d.df, 
    root = d.root,
    split = SplitMetadata('original_split', 'train'),
    transform=transform
)

query_dataset = WildlifeDataset(
    metadata = d.df, 
    root = d.root,
    split = SplitMetadata('original_split', 'test'),
    transform=transform
)

In [85]:
from wildlife_tools.features.deep import DeepFeatures
from wildlife_tools.similarity.cosine import CosineSimilarity
from wildlife_tools.evaluation.classifier import KnnClassifier
import timm

name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))

matcher = CosineSimilarity()
similarity = matcher(query=extractor(query_dataset), database=extractor(reference_dataset))
preds = KnnClassifier(k=1, database_labels=reference_dataset.labels_string)(similarity['cosine'])

acc = sum(preds == query_dataset.labels_string) / len(preds)
print(acc)

 71%|████████████████████████████████████████████▍                  | 24/34 [04:31<01:53, 11.32s/it]


KeyboardInterrupt: 

In [76]:
# Save the features
import os
import torch
os.makedirs('features', exist_ok=True)

torch.save(query, 'features/query.pt')
torch.save(database, 'features/database.pt')

In [78]:
from wildlife_tools.similarity.cosine import CosineSimilarity

similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)

RuntimeError: [enforce fail at alloc_cpu.cpp:117] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 30726021988 bytes. Error code 12 (Cannot allocate memory)